In [ ]:
import re
import random

class TeyvatEliza:
    def __init__(self):
        # Правила: (регулярка, список ответов в стиле Genshin)
        self.rules = [
            # 1. Грусть / усталость / депрессия
            (r".*\b(ГРУЩУ|ПЕЧАЛЮСЬ|УСТАЛ|ДЕПРЕССИЯ|ПЛОХО|ГРУСТНО|РАССТРОЕН|ПЛАЧУ)\b.*", [
                "Ветер Мондштадта тоже иногда плачет... Что заставляет твоё сердце тосковать, Путешественник?",
                "Даже Архонты знают боль утраты. Расскажи, какая тень легла на твою душу?",
                "Тьма Абисса шепчет о печали... Но звёзды всё ещё светят. Что гнетёт тебя?",
                "Листья Инадзумы падают, как слёзы... Чем я могу облегчить твою ношу?"
            ]),

            # 2. "ВСЕГДА" / "НИКОГДА"
            (r".*\b(ВСЕГДА|НИКОГДА)\b.*", [
                "В Тейвате нет ничего вечного, кроме времени Архонтов... Что именно происходит всегда?",
                "Даже небеса Ли Юэ когда-то изменятся... Расскажи подробнее об этом 'всегда'.",
                "Слова 'никогда' опасны, как проклятие Дендро Архонта... А что, если всё-таки возможно?"
            ]),

            # 3. "МНЕ НУЖНО" / "ХОЧУ"
            (r".*\b(МНЕ НУЖНО|Я ХОЧУ)\b\s+(.+)", [
                "Желания — это искры, из которых рождаются видения... Зачем тебе \\2?",
                "В Сумеру говорят: желание рождает знание. Что даст тебе \\2?",
                "Паймон бы сказала: «Опять тратить примогемы?!»... Но серьёзно — зачем тебе \\2?"
            ]),

            # 4. Персонажи / отношения / фандом
            (r".*\b(РАЙДЕН|ДЖИНН|ДИЛЮК|КАЭДЭХАРА|ХУТАО|ЯЭ|АЛЬХАЙТАМ|ЗЛОЙ|ЛЮБЛЮ|НРАВИТСЯ|ХОЧУ|ПРОКАЧАТЬ|ГАЧА)\b.*", [
                "Многие сердца Тейвата бьются в унисон с этим именем... Что связывает тебя с \\1?",
                "Судьба сводит странников с избранными... Расскажи, почему \\1 так важен для тебя?",
                "В Инадзуме говорят: вечность — это любовь... Твои чувства к \\1 вечны?"
            ]),

            # 5. "ПОТОМУ ЧТО"
            (r".*\bПОТОМУ ЧТО\b\s+(.+)", [
                "Причины в Тейвате часто скрыты за семью печатями... Это действительно главная причина?",
                "Даже Царица не всегда говорит правду... Есть ли что-то ещё за этим \\1?",
                "Ветер доносит шепот истины... Но иногда он лжёт. Ты уверен в \\1?"
            ]),

            # 6. Гacha / фарм / игра
            (r".*\b(ГАЧА|ПРИМОГЕМЫ|КРУТКИ|ПРОИГРАЛ|НЕ ВЫПАЛ|ФАРМ|АРТЕФАКТЫ|ДОЛГО|СКУЧНО|НАДОЕЛО)\b.*", [
                "Судьба — жестокая госпожа... Сколько круток унесла бездна на этот раз?",
                "Даже Путешественник не всегда получает желаемое с первого раза... Что ты хочешь выбить?",
                "Фарм артефактов — это ритуал... Но иногда боги глухи. Как долго ты уже мучаешься?"
            ]),

            # 7. Вопросы "ПОЧЕМУ"
            (r".*\bПОЧЕМУ\b.*", [
                "Почему звёзды падают? Почему Архонты молчат? Почему ты спрашиваешь именно это?",
                "Ответы Тейвата часто рождаются в вопросах... Что подсказывает твоё сердце?",
                "Даже Небесный Принцип не отвечает на все 'почему'... А ты сам знаешь ответ?"
            ]),

            # 8. Любой вопрос про персонажа / лор
            (r".*\b(КТО ТАКОЙ|ЧТО ТАКОЕ|ПОЧЕМУ|ЗАЧЕМ|КАК|ГДЕ)\s+(РАЙДЕН|АБИСС|АРХОНТ|ФАТУИ|КЕЦИН|ДРАКОН|ЦАРИЦА|НЕБЕСНЫЙ|ПРИНЦИП)\b.*", [
                "Лор Тейвата глубже, чем бездна... Что именно ты хочешь понять о \\2?",
                "Истина скрыта за завесой... Расскажи, какой кусочек пазла ты ищешь?",
                "Даже мудрейшие из Семи не знают всего... Но я послушаю твои мысли."
            ])
        ]

        # Ответы по умолчанию (когда ничего не подошло)
        self.fallbacks = [
            "Ветер приносит обрывки чужих историй... Продолжай, странник.",
            "Тейват слушает... Что ещё тревожит твою душу?",
            "Паймон бы уже заснула... но я всё ещё здесь. Говори.",
            "Под небом Тейвата каждое слово имеет значение... Расскажи больше.",
            "Звёзды мерцают... они тоже ждут твоего следующего слова.",
            "Путешествие длинное... не торопись. Что у тебя на сердце?"
        ]

    def reflect_pronouns(self, text: str) -> str:
        """Отражение местоимений (я → ты, мой → твой и т.д.)"""
        replacements = [
            (r'\bЯ\b', 'ТЫ'),
            (r'\bМЕНЯ\b', 'ТЕБЯ'),
            (r'\bМНЕ\b', 'ТЕБЕ'),
            (r'\bМОЙ\b', 'ТВОЙ'),
            (r'\bМОЯ\b', 'ТВОЯ'),
            (r'\bМОЁ\b', 'ТВОЁ'),
            (r'\bМОИ\b', 'ТВОИ'),
            (r'\bМЫ\b', 'ВЫ'),
            (r'\bНАШ\b', 'ВАШ'),
        ]

        result = text
        for pat, repl in replacements:
            result = re.sub(pat, repl, result, flags=re.IGNORECASE)
        return result

    def get_response(self, user_input: str) -> str:
        """Основная логика ответа"""
        if not user_input:
            return "Тишина Тейвата красноречивее слов... Но всё же — говори."

        text_upper = user_input.upper().strip()

        # Выход
        if text_upper in ['ВЫХОД', 'ПОКА', 'EXIT', 'QUIT', 'ДО СВИДАНИЯ']:
            return "Пусть семь элементов хранят твой путь. До новых встреч, Путешественник."

        # Поиск подходящего правила
        for pattern, answers in self.rules:
            match = re.search(pattern, text_upper)
            if match:
                resp = random.choice(answers)
                # Подставляем группы захвата
                for i, group in enumerate(match.groups(), 1):
                    if group:
                        resp = resp.replace(f'\\{i}', group)
                return resp

        # Отражение + дефолт
        if len(text_upper.split()) > 2:
            reflected = self.reflect_pronouns(text_upper)
            return f"{reflected}... Так шепчет ветер Тейвата."

        return random.choice(self.fallbacks)


def main():
    print("=" * 70)
    print("✨ ДОБРО ПОЖАЛОВАТЬ В ТЕЙВАТ ✨")
    print("Я — странствующий дух, слушатель судеб и хранитель тайн Архонтов.")
    print("Поделись своими мыслями, Путешественник...")
    print("(Напиши 'выход' или 'пока', чтобы завершить разговор)")
    print("=" * 70)

    bot = TeyvatEliza()

    while True:
        try:
            msg = input("\n👤 Ты: ").strip()
            if not msg:
                print("✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.")
                continue

            reply = bot.get_response(msg)
            print(f"✨ Дух: {reply}")

            if "До новых встреч" in reply:
                break

        except (KeyboardInterrupt, EOFError):
            print("\n✨ Дух: Пусть звёзды освещают твой путь... До встречи в Тейвате.")
            break


if __name__ == "__main__":
    main()

✨ ДОБРО ПОЖАЛОВАТЬ В ТЕЙВАТ ✨
Я — странствующий дух, слушатель судеб и хранитель тайн Архонтов.
Поделись своими мыслями, Путешественник...
(Напиши 'выход' или 'пока', чтобы завершить разговор)
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ... Но слова помогают найти путь.
✨ Дух: Тишина — тоже ответ.